In [2]:
import pandas as pd

diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000
mean,3.848329,120.822622,69.035990,20.457584,79.521851,31.982262,0.470871,33.317481,0.344473
std,3.360782,31.883264,19.432323,15.954452,114.862405,7.853917,0.330669,11.816300,0.475502
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.241500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,27.000000,32.000000,0.370000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,126.750000,36.500000,0.625500,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [3]:
diabetes_un = diabetes.drop_duplicates()
print(diabetes.shape)
print(diabetes_un.shape)

(778, 10)
(768, 10)


In [5]:
def get_low_information_cols(data, porog=0.95):
    low_information_cols = []
    for col in data.columns:
        top_freq = data[col].value_counts(normalize=True).max()
        nunique_ratio = data[col].nunique() / data[col].count()
        if top_freq > porog:
            low_information_cols.append(col)
            # print(f'{col}: {round(top_freq*100,2)}% одинаковых значений')
        if nunique_ratio > porog:
            low_information_cols.append(col)
            # print(f'{col}: {round(nunique_ratio*100,2)}% уникальных значений')
    return low_information_cols

print(get_low_information_cols(diabetes_un))

['Gender']


In [9]:
diabetes_un2 = diabetes_un.drop('Gender', axis=1)
print(diabetes_un.shape, diabetes_un2.shape)

(768, 10) (768, 9)


In [13]:
diabetes_un2[diabetes_un2['Insulin'].isnull()].count()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64